In [ ]:
using Dates, CSV, JuliaDB, Plots, PlotThemes, Interact, 
    HTTP, SingularSpectrumAnalysis, ProgressMeter
using OnlineStats
theme(:bright)

# OnlineStats and Big Data Viz

## `Partition` and `IndexedPartition`

- Plot any-sized dataset.
- Here are two plots with **100 Million observations**:

In [ ]:
o = Partition(Series(Mean(), Extrema()))

y = randn()

@showprogress for _ in 1:10^8
    fit!(o,  global y += randn())
end

plot(o)

In [ ]:
o = IndexedPartition(Float64, KHist(10))

@showprogress for _ in 1:10^8
    fit!(o,  (randn(), randn()))
end

plot(o)

<br><br><br><br><br><br><br><br><br><br><br><br><br><br>

# Kaggle's Huge Stock Market Dataset

- Source: https://www.kaggle.com/borismarjanovic/price-volume-data-for-all-us-stocks-etfs
- OHLC data for each stock/ETF (each is a separate CSV) traded in the US
- Just over 700MB

In [ ]:
path = "/Users/joshday/datasets/price-volume-data-for-all-us-stocks-etfs/Stocks/"

readdir(path)

In [ ]:
;head "$path/aapl.us.txt"

## Statistics/Plots Directly From CSV

- Via **`CSV.File`**

In [ ]:
function plot_high_low(stock; kw...)
    o = IndexedPartition(Date, Extrema(), 500)
    for row in CSV.File("$path/$stock.us.txt") 
        fit!(o, [row.Date => row.Low, row.Date => row.High])
    end
    t = uppercase(stock) * " (nobs = $(nobs(o)))"
    plot(o; xlab="Date", title=t, legend=false, kw...)
end

In [ ]:
plot_high_low("aapl")

<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>

## ...But this loads the entire CSV!

Even though we are creating the plots with OnlineStats, the entire file is loaded into memory.

### `CSV.Rows`

- `CSV.Rows` lets you lazily read from a CSV file
    - **Minimal memory footprint**
    - At the cost of **no type inference** (everything is treated as `String`)

In [ ]:
function plot_high_low2(stock; kw...)
    o = IndexedPartition(Date, Extrema(), 500)
    for row in CSV.Rows("$path/$stock.us.txt")
        dt = Date(row.Date, "yyyy-mm-dd")
        low = parse(Float64, row.Low)
        hi = parse(Float64, row.High)
        fit!(o, [dt => low, dt => hi])
    end
    t = uppercase(stock) * " (nobs = $(nobs(o)))"
    plot(o; xlab="Date", title=t, legend=false, kw...)
end

In [ ]:
plot_high_low2("aapl")

In [ ]:
plot(plot_high_low2("aapl"), plot_high_low2("aapl"), layout=(2,1), link=:x)

<br><br><br><br><br><br><br><br><br><br><br><br><br>

## Loading Multiple Datasets at Once

- Working with only one CSV at a time limits what kinds of analyses we can do.

In [ ]:
t = loadtable(path, filenamecol = :Stock)

In [ ]:
table(t, pkey=(1,2))

In [ ]:
o = CovMatrix()

fit!(o, select(t, (:Open, :High)))

In [ ]:
plot(o)

### Passing an `OnlineStat` as a reducer

In [ ]:
reduce(Mean(), t, select=(:Open, :Close) => x -> -(x...))

### `groupreduce`

- Like `groupby`, but **much** more memory efficient!

In [ ]:
temp = groupreduce(Mean(), t, :Stock, select=:Open => x -> x + 1)
select(temp, (1, 2 => value))

### Make All Plots at Once

In [ ]:
allplots = groupreduce(IndexedPartition(Date, Extrema()), t, :Stock, select=(:Date, :Close))

In [ ]:
@manipulate for s in ["aapl", "msft", "ibm", "googl", "nflx"]
    t2 = filter(==("$s.us.txt"), allplots, select=:Stock)
    plot(t2[1].IndexedPartition, title=s)
end

<br><br><br><br><br><br><br><br><br><br><br><br><br>

## Time Series Analysis Tools

### Singular Spectrum Analysis

- Extract seasonal components out of a time series
- Calculated "offline", but a useful tool

In [ ]:
temp = filter(==("aapl.us.txt"), t, select=:Stock)

yt, ys = analyze(select(temp, :Close), 90)
plot(yt, lab="Trend")
plot!(ys, lab="Season")

### Autocorrelation

In [ ]:
temp = filter(==("aapl.us.txt"), t, select=:Stock)


plot(fit!(AutoCov(365), select(temp, :Close)))

<br><br><br><br><br><br><br><br><br><br><br><br><br><br>
# TrueFX API

In [ ]:
function get_data(q = "")
    endpoint = "https://webrates.truefx.com/rates/connect.html?f=csv&$q"
    hdr = [:pair, :utc, :big_bid_figure, :bid_points, :offer_bid_figure,
           :offer_points, :high, :low, :open]
    r = HTTP.get(endpoint)
    CSV.read(r.body; header=hdr, footerskip=1)
end

In [ ]:
get_data("c=EUR/USD")

In [ ]:
o = IndexedPartition(Int, Extrema(), 10)

while true
    IJulia.clear_output(true)
    df = get_data("c=USD/JPY")
    fit!(o, (df[1, :utc], df[1, :bid_points]))
    display(plot(o, xlab="UTC Time", ylab="Bid Points", title="nobs = $(nobs(o))"))
end

<br><br><br><br><br><br><br><br><br><br>
## NYC Yellow Taxi Data (2018)

- Source: https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page
- Data on every single yellow taxi trip in 2018!

In [ ]:
datapath = "/Users/joshday/datasets/nyc_yellow_taxi_2018"

sum(filesize, ("$datapath/$file") for file in readdir(datapath)) / 1024^3

In [ ]:
;head "$datapath/yellow_tripdata_2018-01.csv"

In [ ]:
passenger_counts = Partition(CountMap(Int), 365)
@showprogress for file in readdir(datapath), row in CSV.Rows("$datapath/$file", skipto=3)
    fit!(passenger_counts, parse(Int, row.passenger_count))
end
plot(passenger_counts)

### In Parallel

In [ ]:
using Distributed
addprocs()

@everywhere begin 
    datapath = "/Users/joshday/datasets/nyc_yellow_taxi_2018"
    using Dates, OnlineStats, CSV
end 

In [ ]:
@time begin
    gb = @distributed merge for file in readdir(datapath)
        stat = GroupBy(Date, CountMap(Int))
        for row in CSV.Rows("$datapath/$file", skipto=3)
            fit!(stat, Date(row.tpep_pickup_datetime[1:10]) => parse(Int, row.passenger_count))
        end
        stat
    end
end
nothing

In [ ]:
@manipulate for day in Date(2018,1,1):Day(1):Date(2018,12,31)
    plot(gb.value[day], title=string(day))
end